In [45]:
from typing import List, Dict, Any

import pandas as pd

from debiasing_cot import DeCoT
from relaxed_fda import RelaxedFDA
from reasoning_path_builder import ReasoningPath
from doraemon import Doraemon


logger = Doraemon.get_logger(name=__name__, logfile=f"decot_on_hotpotqa.log")

file_cot='/kaggle/input/llama-3-building-cots-on-musique/cots_df.pkl'
file_sot='/kaggle/input/llama-3-building-sots-on-musique/sots_df.pkl'

df_cot=pd.read_pickle(file_cot)

df_sot=pd.read_pickle(file_sot)

import tiktoken

enc = tiktoken.encoding_for_model("gpt-3.5-turbo")  # or "gpt-4"

def count_tokens(text):
    return len(enc.encode(text))

In [ ]:
logger.info(df_cot.shape)
df_cot['num_tokens'] = df_cot['r_s'].apply(lambda x: count_tokens(x))
avg_tokens = df_cot['num_tokens'].mean()
logger.info(avg_tokens)

logger.info(df_cot.iloc[0].r_s)

In [ ]:
logger.info(df_sot.shape)
df_sot['num_tokens'] = df_sot['r_s'].apply(lambda x: count_tokens(x))
avg_tokens = df_sot['num_tokens'].mean()
logger.info(avg_tokens)

logger.info(df_sot.iloc[0].r_s)

In [53]:
import re
import pandas as pd

def count_steps_sot(text):
    """Count steps for <think>...</think> format (SoT)."""
    if not isinstance(text, str):
        return None
    match = re.search(r'<think>(.*?)</think>', text, re.DOTALL)
    if match:
        inner = match.group(1)
        return len([seg for seg in inner.split('→') if seg.strip()])
    return None

def count_steps_cot(text):
    """Count steps for natural language CoT format."""
    if not isinstance(text, str):
        return None
    main = re.split(r'\\boxed{|\\boxed{|A:|Answer:', text)[0]
    sentences = re.split(r'(?<=[.?!])\s+', main)
    return len([s for s in sentences if s.strip()])

# Example usage
# For SoT-formatted DataFrame:
df_sot['sot_steps'] = df_sot['r_s'].apply(count_steps_sot)
logger.info(df_sot['sot_steps'].mean())

# For CoT-formatted DataFrame:
df_cot['cot_steps'] = df_cot['r_s'].apply(count_steps_cot)
logger.info(df_cot['cot_steps'].mean())

2025-07-03 07:19:11,060 INFO 4.732134624250807
2025-07-03 07:19:11,722 INFO 5.983815137056787


In [61]:
df=pd.read_pickle('/kaggle/input/generate-alter-decot-on-musiqueqa/alternative_decots.pkl')
df['cot_steps'] = df['cots'].apply(lambda x: count_steps_cot(x[0]))
logger.info(df['cot_steps'].mean())

2025-07-03 07:27:20,974 INFO 5.5547201336675025
